In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
#  Rescue-profile breed lists
water_breeds   = ["Labrador Retriever Mix",
                  "Chesapeake Bay Retriever",
                  "Newfoundland"]

mountain_breeds = ["German Shepherd",
                   "Alaskan Malamute",
                   "Old English Sheepdog",
                   "Siberian Husky",
                   "Rottweiler"]

disaster_breeds = ["Doberman Pinscher",
                   "German Shepherd",
                   "Golden Retriever",
                   "Bloodhound",
                   "Rottweiler"]

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='assets/grazioso_logo.png', style={'height':'80px'}),
    html.H3("Student: 2569281"),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
    html.Label("Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset (All Dogs)',           'value': 'reset'},
            {'label': 'Water Rescue',               'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster / Tracking',        'value': 'disaster'}
        ],
        value='reset',
        labelStyle={'display': 'block'}
    )
]),

    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id', 'data'),
              Input('filter-type', 'value'))
def update_dashboard(filter_type):

    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": water_breeds},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    elif filter_type == 'mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": mountain_breeds},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": disaster_breeds},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    else:                     # reset
        query = {"animal_type": "Dog"}

    dff = pd.DataFrame.from_records(db.read(query))
    dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    counts = dff['breed'].value_counts().reset_index()
    counts.columns = ['breed', 'total']
    fig = px.bar(counts, x='breed', y='total',
                 title='Outcome Count by Breed')
    fig.update_layout(height=350, margin=dict(l=20, r=20, t=50, b=20))
    return [dcc.Graph(figure=fig)]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [
        {
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:                # No data yet
        return []

    dff = pd.DataFrame(viewData)
    if dff.empty:
        return []

    # Pick a safe row index
    row = index[0] if index else 0
    if row >= len(dff):
        row = 0

        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(mode="external", debug=True, port=8060)

Dash app running on http://127.0.0.1:8060/
